# **Preparation Notebook**



---
## Setup Environment

In [1]:
# DO NOT MODIFY THE CODE IN THIS CELL
!pip install -q utstd

from utstd.folders import *
from utstd.ipyrenders import *

at = AtFolder(
    course_code=36106,
    assignment="AT2",
)
at.run()

import warnings
warnings.simplefilter(action='ignore')


[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



You can now save your data files in: c:\Users\brohao\Desktop\UTS\36106\ClassificationModels-36106-AT2\36106\assignment\AT2\data


---
## Student Information

In [ ]:
# <Student to fill this section and then remove this comment>
student_name = "Jiayu Hao"
student_id = "25948860"

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h1", key='student_name', value=student_name)

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h1", key='student_id', value=student_id)

---
## 0. Python Packages

### 0.a Install Additional Packages

> If you are using additional packages, you need to install them here using the command: `! pip install <package_name>`

In [ ]:
# <Student to fill this section and then remove this comment>

### 0.b Import Packages

In [2]:
# DO NOT MODIFY THE CODE IN THIS CELL
import pandas as pd
import altair as alt

---
## A. Feature Selection


## A.0 # Load Data

In [3]:
# DO NOT MODIFY THE CODE IN THIS CELL
try:
  df = pd.read_csv(at.folder_path / "credit_rating.csv")
except Exception as e:
  print(e)

### A.1 Approach 1 — Remove PII & non-causal address blocks

In [4]:
# <Student to fill this section and then remove this comment>
pii_cols = [
    "customer_id","prefix","full_name","email","phone_number",
    "secondary_address","building_number","street_name","street_suffix",
    "city","postcode","state_abbr","dob","cc_number","cc_expiry","cc_security_code"
]

In [ ]:
# provide an explanation on why you use this approach for feature selection and describe its results
feature_selection_1_insights = """
Why（为什么）：去除不用于预测、且可能带来偏见或数据泄露风险的字段（PII，如姓名/邮箱/卡号等），并删除明显无关的地址型特征，提升可解释性与合规性。
How（怎么做）：基于数据字典，显式列出要丢弃的列。
Result（结果）：降低维度、减少噪声，为后续建模留存“业务可用的、可解释的”变量。
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_selection_1_insights', value=feature_selection_1_insights)

### A.2 Approach 2 — Leakage / quasi-labels

In [5]:
# <Student to fill this section and then remove this comment>
potential_leak_cols = [
    "cc_interest_rate",     # 可能与风险定价强相关
    "credit_limit_change",  # 额度调整常受评级/策略影响-可能存在捷径效应-对照
    "credit_mix"            # 与目标标签体系近似（准标签风险-伦理/泛化风险）-对照
    "gender"
]

In [ ]:
# provide an explanation on why you use this approach for feature selection and describe its results
feature_selection_2_insights = """
Why：避免把“未来信息/强泄漏”或“重复编码”的变量放进模型（例如：如果存在与目标强同义的特征，或相同语义的月/年历史并存）。
How：

“信用历史年限”选择 count_credit_history_years，若有月度版则只留一个（你的数据里仅有 years）。

信用卡利率、卡号等与风险弱相关或合规风险较高：全部剔除。
Result：降低隐形泄漏风险与共线冗余。
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_selection_2_insights', value=feature_selection_2_insights)

### A.3 Approach3 — Keep core signals + lightweight context

> You can add more cells related to other approaches in this section

In [ ]:
# <Student to fill this section and then remove this comment>

In [12]:
base_features = [
    # demographics / context
    "age","birth_country","occupation",
    # ability / behavior
    "annual_income","count_bank_accounts","count_credit_cards",
    "loans_count","avg_days_past_due","count_delayed_payment",
    "count_credit_inquiries","outstanding_debt","credit_ratio",
    "count_credit_history_years","monthly_emi_payment","min_amount_payment",
    "_monthly_invested_amount","payment_behaviour","monthly_balance",
    # low-dimensional loan-type context
    "last_1_loan_type","last_2_loan_type","last_3_loan_type","last_4_loan_type",
    "last_5_loan_type","last_6_loan_type","last_7_loan_type","last_8_loan_type","last_9_loan_type"
]


In [ ]:
# provide an explanation on why you use this approach for feature selection and describe its results
feature_selection_n_insights = """
Why：结合你给出的 EDA/业务建议与多分类信用评级经验，保留强区分度变量，并保留少量上下文（职业、信用组合、支付行为、最近一次贷款类型）。
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_selection_n_insights', value=feature_selection_n_insights)

### A.z Final Selection of Features

In [15]:
target_name = "credit_rating"

In [16]:
# <Student to fill this section and then remove this comment>
drop_cols = set(pii_cols + potential_leak_cols)
features_list = base_features + [target_name]

In [17]:
print("Mainline kept features:", len(features_list))

Mainline kept features: 28


In [ ]:
# provide a quick explanation on the features selected
feature_selection_explanations = """
删除 PII/地址/卡号/安全码/到期等敏感或弱相关字段，满足伦理与合规。

保留与信用风险最相关的交易/负债/历史/行为特征，并添加少量上下文（职业、最近一次贷款类型）。
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_selection_explanations', value=feature_selection_explanations)

---
## B. Data Cleaning

In [24]:
# DO NOT MODIFY THE CODE IN THIS CELL
# Create copy of datasets
try:
  df_clean = df[features_list].copy()
except Exception as e:
  print(e)

### B.1 Fixing invalid values

In [25]:
# Negative overdue days -> 0
df_clean["avg_days_past_due"] = df_clean["avg_days_past_due"].clip(lower=0)

# Negative EMI -> 0
df_clean["monthly_emi_payment"] = df_clean["monthly_emi_payment"].clip(lower=0)

In [ ]:
# Provide some explanations on why you believe it is important to fix this issue and its impacts
data_cleaning_1_explanations = """
Why：avg_days_past_due 理应非负；monthly_emi_payment 理应非负。负值会误导模型。
Impact：修复后分布更符合业务含义，减少异常对模型的影响。
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_cleaning_1_explanations', value=data_cleaning_1_explanations)

### B.2 Fixing missing values

In [ ]:
# <Student to fill this section and then remove this comment>
# 存在missing value？

In [ ]:
# 没跑过！没问题的话最后删掉
cat_cols = [c for c in df_clean.columns if df_clean[c].dtype == "object" and c != target_name]
num_cols = [c for c in df_clean.columns if c != target_name and np.issubdtype(df_clean[c].dtype, np.number)]

for c in cat_cols:
    df_clean[c] = df_clean[c].fillna("Unknown")
for c in num_cols:
    df_clean[c] = df_clean[c].fillna(df_clean[c].median())

In [ ]:
# Provide some explanations on why you believe it is important to fix this issue and its impacts
data_cleaning_2_explanations = """
可能需要删掉！！！
Why：虽然你给出的概览显示“无缺失”，但为鲁棒性仍添加缺失处理，避免后续数据增强或新样本报错。
Impact：保证生产可用性与 Notebook 可复现性。
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_cleaning_2_explanations', value=data_cleaning_2_explanations)

### B.3 Fixing Ratio caps

In [26]:
# <Student to fill this section and then remove this comment>
df_clean["credit_ratio"].describe()

count    12500.000000
mean        32.289001
std          5.119387
min         20.880082
25%         28.010949
50%         32.357246
75%         36.504832
max         48.176599
Name: credit_ratio, dtype: float64

In [27]:
# credit_ratio 视为百分比 -> 小数；截断在 [0, 1]
df_clean["credit_ratio"] = (df_clean["credit_ratio"] / 100.0)

In [ ]:
# Provide some explanations on why you believe it is important to fix this issue and its impacts
data_cleaning_3_explanations = """
Why：credit_ratio, 之后会创建的 dti 与 emi_to_income 可能受极端值影响；轻量截断可稳健训练。
Impact：减少长尾对线性模型与距离度量的冲击。
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_cleaning_3_explanations', value=data_cleaning_3_explanations)

### B.n Fixing Unit

> You can add more cells related to other issues in this section

In [28]:
# <Student to fill this section and then remove this comment>
df_clean["count_credit_history_years"].describe()

count    12500.000000
mean       221.158240
std         99.702928
min          2.000000
25%        144.750000
50%        219.000000
75%        302.000000
max        404.000000
Name: count_credit_history_years, dtype: float64

In [29]:
# 信用历史：把“count_credit_history_years（实为月）”校正成年，并删除原列
if "count_credit_history_years" in df_clean.columns:
    df_clean["credit_history_years"] = (df_clean["count_credit_history_years"] / 12.0).clip(lower=0)
    df_clean.drop(columns=["count_credit_history_years"], inplace=True)

In [30]:
df_clean["credit_history_years"].describe()

count    12500.000000
mean        18.429853
std          8.308577
min          0.166667
25%         12.062500
50%         18.250000
75%         25.166667
max         33.666667
Name: credit_history_years, dtype: float64

In [ ]:
# Provide some explanations on why you believe it is important to fix this issue and its impacts
data_cleaning_n_explanations = """
count_credit_history_years 的单位：观察到均值≈221、最大≈404，按“年”显然不合理，按“月”改成年（/12）是正确的.
为避免共线冗余，新增 credit_history_years = count_credit_history_years/12 并删除原列。
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_cleaning_n_explanations', value=data_cleaning_n_explanations)

---
## C. Feature Engineering

In [49]:
# DO NOT MODIFY THE CODE IN THIS CELL
# Create copy of datasets

try:
  df_eng = df_clean.copy()
except Exception as e:
  print(e)

### C.1 New Feature "dti"



In [44]:
# <Student to fill this section and then remove this comment>
eps = 1e-6

In [50]:
df_eng["dti"] = (df_eng["outstanding_debt"].clip(lower=0) * 12.0) / (df_eng["annual_income"].abs() + eps)

In [ ]:
# Provide some explanations on why you believe it is important to create this feature and its impacts
feature_engineering_1_explanations = """
Why：你指出 DTI 需要 (outstanding_debt * 12) / annual_income；该比率能反映还款压力。
Impact：对“Good/Standard/Poor”区分常见有效；可与 outstanding_debt 二选一进入模型，降低共线性。
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_1_explanations', value=feature_engineering_1_explanations)

### C.2 New Feature "emi"



In [51]:
df_eng["emi_to_income"] = (df_eng["monthly_emi_payment"] * 12.0) / (df_eng["annual_income"].abs() + eps)

In [ ]:
# Provide some explanations on why you believe it is important to create this feature and its impacts
feature_engineering_2_explanations = """
Why：月供占收入越高，违约/评级下调风险越高。
Impact：补充还款压力维度，与 DTI 共同反映“现金流紧张度”。
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_2_explanations', value=feature_engineering_2_explanations)

### C.3 New Feature Log1p for skewed amounts



In [35]:
# <Student to fill this section and then remove this comment>

In [47]:
for c in ["annual_income","outstanding_debt","monthly_emi_payment","_monthly_invested_amount","monthly_balance"]:
    df_eng[f"log1p__{c}"] = np.log1p(df_eng[c])

In [ ]:
# Provide some explanations on why you believe it is important to create this feature and its impacts
feature_engineering_3_explanations = """
Why：金额/余额等分布长尾，log1p 稳健；对多数模型更友好。
Impact：提升线性可分性与稳定性。
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_3_explanations', value=feature_engineering_3_explanations)

### C.4 New Feature Loan-type low-dimensional signals



In [ ]:
# <Student to fill this section and then remove this comment>

In [53]:
# 1. 找出所有贷款类型的列（以 last_ 开头，以 _loan_type 结尾）
loan_cols = [c for c in df.columns if c.startswith("last_") and c.endswith("_loan_type")]
print("Loan type columns:", loan_cols)

# 2. 合并所有贷款列为一列（长表形式），方便统计
loan_types_all = pd.Series(df[loan_cols].values.ravel())

# 3. 去掉缺失值（如果有的话）
loan_types_all = loan_types_all.dropna()

# 4. 统计每种贷款类型的数量
loan_type_counts = loan_types_all.value_counts()

# 5. 打印结果
print("Unique loan types:", loan_type_counts.shape[0])
print("\nLoan type counts:")
print(loan_type_counts)

Loan type columns: ['last_9_loan_type', 'last_8_loan_type', 'last_7_loan_type', 'last_6_loan_type', 'last_5_loan_type', 'last_4_loan_type', 'last_3_loan_type', 'last_2_loan_type', 'last_1_loan_type']
Unique loan types: 10

Loan type counts:
No Loan                    68339
Payday Loan                 5071
Credit-Builder Loan         5055
Not Specified               4952
Home Equity Loan            4888
Student Loan                4871
Mortgage Loan               4867
Personal Loan               4861
Debt Consolidation Loan     4847
Auto Loan                   4749
Name: count, dtype: int64


In [54]:
df_eng["flag_payday_recent"] = (
    df_eng[loan_cols].apply(lambda row: int(any(str(v) == "Payday Loan" for v in row)), axis=1)
)

# 是否“有过贷款”（不全是 'No Loan'）
df_eng["flag_has_any_loan"] = (
    df_eng[loan_cols].apply(lambda row: int(any(str(v) != "No Loan" for v in row)), axis=1)
)

# 多样性（不同贷款类型的去重计数；'No Loan' 不计入）
def _diversity(xs):
    s = set([str(v) for v in xs if str(v) != "No Loan"])
    return len(s)
df_eng["loan_type_diversity"] = df_eng[loan_cols].apply(_diversity, axis=1)

In [ ]:
# Provide some explanations on why you believe it is important to create this feature and its impacts
feature_engineering_3_explanations = """
由于 ‘No Loan’ 占绝大多数，我们不直接 one-hot，而是做了低维表征。
贷款类型降维为 3 个强信号（是否有/是否Payday/多样性），减少变量控制维度又保留信息，为后续的onehot做准备，提升可解释性
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_3_explanations', value=feature_engineering_3_explanations)

### C.n Fixing Multicollinearity

> You can add more cells related to new features in this section

In [65]:
# <Student to fill this section and then remove this comment>
try:
    import statsmodels.api as sm  # just to test availability
except Exception:
    !pip install statsmodels

In [66]:
# VIF 需要的函数
def _safe_import_vif():
    try:
        from statsmodels.stats.outliers_influence import variance_inflation_factor
        return variance_inflation_factor
    except Exception as e:
        print("Failed to import VIF from statsmodels:", e)
        return None

def compute_collinearity_report(
    df_numeric: pd.DataFrame,
    corr_threshold: float = 0.90,
    vif_threshold: float = 10.0,
    verbose: bool = True,
):
    """
    共线性报告 / Collinearity report
    - Spearman相关矩阵（对偏态稳健）
    - 高相关特征对（|rho| >= corr_threshold）
    - VIF >= vif_threshold 的特征

    Parameters
    ----------
    df_numeric : DataFrame
        仅包含数值列的 DataFrame（已处理缺失/异常）。Only numeric columns.
    corr_threshold : float
        Spearman 绝对相关阈值。
    vif_threshold : float
        VIF 阈值（常见阈值：5或10）。
    verbose : bool
        是否打印结果。

    Returns
    -------
    corr_spearman : DataFrame  # Spearman相关矩阵
    high_corr_pairs : list[(col_i, col_j, rho)]
    vif_df : DataFrame         # 各列的VIF值
    high_vif_cols : list[str]  # VIF超阈值的列名
    """
    # 1) Spearman correlation
    corr_spearman = df_numeric.corr(method="spearman")

    # 找到高相关对（上三角）
    high_corr_pairs = []
    cols = corr_spearman.columns.tolist()
    for i in range(len(cols)):
        for j in range(i+1, len(cols)):
            rho = corr_spearman.iloc[i, j]
            if np.abs(rho) >= corr_threshold:
                high_corr_pairs.append((cols[i], cols[j], float(rho)))

    # 2) VIF
    vif_func = _safe_import_vif()
    vif_df = None
    high_vif_cols = []
    if vif_func is not None and df_numeric.shape[1] >= 2:
        # 标准化可提升数值稳定性（VIF对尺度不敏感，但有助于数值条件）
        X = (df_numeric - df_numeric.mean()) / (df_numeric.std(ddof=0) + 1e-12)
        X = X.replace([np.inf, -np.inf], np.nan).fillna(0.0)

        vif_values = []
        for k, col in enumerate(X.columns):
            try:
                vif_k = vif_func(X.values, k)
            except Exception as e:
                vif_k = np.nan
            vif_values.append(vif_k)
        vif_df = pd.DataFrame({"feature": X.columns, "VIF": vif_values}).sort_values("VIF", ascending=False)
        high_vif_cols = vif_df.loc[vif_df["VIF"] >= vif_threshold, "feature"].tolist()

    if verbose:
        print(f"[Spearman |corr|>={corr_threshold}] pairs: {len(high_corr_pairs)}")
        for (a, b, r) in sorted(high_corr_pairs, key=lambda x: -abs(x[2]))[:20]:
            print(f"  {a:35s} ~ {b:35s}  rho={r: .3f}")
        if vif_df is not None:
            print("\n[VIF Top 15]")
            display(vif_df.head(15))
            print(f"\n[VIF >= {vif_threshold}] cols: {high_vif_cols}")

    return corr_spearman, high_corr_pairs, vif_df, high_vif_cols

In [67]:
import numpy as np
import pandas as pd
# 取数值列（排除目标列）
target_name = "credit_rating"
num_cols_for_check = [
    c for c in df_eng.columns
    if (c != target_name) and np.issubdtype(df_eng[c].dtype, np.number)
]

df_num = df_eng[num_cols_for_check].copy()

# 可选：按经验先做轻度上/下截断，减少极端值干扰（我们之前已在工程中做过）
# 这里直接用df_num原值即可

# 跑共线性报告
corr_s, high_corr_pairs, vif_df, high_vif_cols = compute_collinearity_report(
    df_numeric=df_num,
    corr_threshold=0.90,  # 你也可以试 0.85 做更严格的筛选
    vif_threshold=10.0,   # 常用阈值：5 或 10
    verbose=True,
)

[Spearman |corr|>=0.9] pairs: 1
  loans_count                         ~ loan_type_diversity                  rho= 0.951

[VIF Top 15]


,feature,VIF
3,loans_count,10.450203
13,loan_type_diversity,9.005824
10,emi_to_income,2.607923
6,count_credit_inquiries,2.088517
8,credit_history_years,2.053730
1,count_bank_accounts,1.936540
4,avg_days_past_due,1.913110
5,count_delayed_payment,1.865375
9,dti,1.555475
12,flag_has_any_loan,1.532989



[VIF >= 10.0] cols: ['loans_count']


In [ ]:
# Provide some explanations on why you believe it is important to create this feature and its impacts
feature_engineering_n_explanations = """
Why：outstanding_debt 与 dti 高相关；在“最终进入建模的特征”时二选一（下面 D.2 会只保留 dti）。
Impact：降低线性模型不稳定性，便于解释。
"""

In [69]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_n_explanations', value=feature_engineering_n_explanations)

### C.n Fixing Multicollinearity（去除原始字段）

> You can add more cells related to new features in this section

In [64]:
# 必要删除：避免共线性 & 特征爆炸
drop_cols = [
    "annual_income",             # 已由 log1p__annual_income + dti + emi_to_income 替代
    "outstanding_debt",          # 已包含在 dti
    "monthly_emi_payment",       # 已包含在 emi_to_income
    "_monthly_invested_amount",  # 已做 log1p__ 版本
    "monthly_balance",           # 已做 log1p__ 版本
] + loan_cols  # 原始 last_k_loan_type 全部替换为低维信号

df_eng.drop(columns=[c for c in drop_cols if c in df_eng.columns], inplace=True)
print("Dropped redundant raw features:", drop_cols)
print(df_eng.columns)

Dropped redundant raw features: ['annual_income', 'outstanding_debt', 'monthly_emi_payment', '_monthly_invested_amount', 'monthly_balance', 'last_9_loan_type', 'last_8_loan_type', 'last_7_loan_type', 'last_6_loan_type', 'last_5_loan_type', 'last_4_loan_type', 'last_3_loan_type', 'last_2_loan_type', 'last_1_loan_type']
Index(['age', 'birth_country', 'occupation', 'count_bank_accounts',
       'count_credit_cards', 'loans_count', 'avg_days_past_due',
       'count_delayed_payment', 'count_credit_inquiries', 'credit_ratio',
       'min_amount_payment', 'payment_behaviour', 'credit_rating',
       'credit_history_years', 'dti', 'emi_to_income', 'flag_payday_recent',
       'flag_has_any_loan', 'loan_type_diversity'],
      dtype='object')


KeyError: ''

In [ ]:
# <Student to fill this section and then remove this comment>
feature_engineering_n_explanations = """
Provide some explanations on why you believe it is important to create this feature and its impacts
避免多重共线性限制，和onehot带来的维度灾难问题，派生变量的原始变量经过评估后被去除。
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_n_explanations', value=feature_engineering_n_explanations)

---
## D. Data Preparation for Modeling

### D.1 Split Datasets


In [ ]:
# <Student to fill this section>

In [55]:
from sklearn.model_selection import train_test_split

X = df_eng.drop(columns=[target_name])
y = df_eng[target_name].copy()

X_train_raw, X_temp_raw, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y  # 70% train
)
X_val_raw, X_test_raw, y_val, y_test = train_test_split(
    X_temp_raw, y_temp, test_size=0.50, random_state=42, stratify=y_temp  # 15%/15%
)

print("Shapes:",
      X_train_raw.shape, X_val_raw.shape, X_test_raw.shape,
      y_train.shape, y_val.shape, y_test.shape)

Shapes: (8750, 32) (1875, 32) (1875, 32) (8750,) (1875,) (1875,)


In [ ]:
# Provide some explanations on what is the best strategy to use for data splitting for this dataset
data_splitting_explanations = """
Strategy（策略）：

Stratified 按 credit_rating 分层，保持类分布稳定（原分布约 Standard 53% / Poor 29% / Good 18%）。

划分比例：70/15/15（train/val/test）。
Why：
部分模型/库的限制：

有些 baseline 模型、快速实验时（例如 Logistic Regression 不调参，或者用默认参数的树模型），可能直接用一个固定 validation set 来评估就足够。

如果强行上 KFold，反而会拖慢迭代速度。

兼容性：

你留 15% 作为固定 validation set，可以确保“所有模型都有统一的比较基准”；

在需要更严谨的场合（调参、模型比较），再在 70% 的训练部分用 KFold cross-validation。

这样就能解释为什么报告里有 “val set” 又有 “KFold”

val set = 给简单/非CV模型一个稳定对比基准；

kfold = 给复杂/调参模型更稳健的评估；

test set = 最后统一检验。

先划分再做编码/标准化（以免信息泄漏）；但此处为输出可直接训练的矩阵，采用仅基于 train 统计量的变换（如 StandardScaler 的 fit 在 train 上，transform 应用于 val/test）。
"""

In [ ]:
# Do not modify this code
print_tile(size="h3", key='data_splitting_explanations', value=data_splitting_explanations)

### D.2 Data Transformation — OHE + Standardize（仅用 train 拟合）


In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

categorical_cols = [c for c in X_train_raw.columns if X_train_raw[c].dtype == "object"]
numeric_cols = [c for c in X_train_raw.columns if c not in categorical_cols]

# One-Hot（忽略未知）
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
scaler = StandardScaler(with_mean=True, with_std=True)

preprocess = ColumnTransformer(
    transformers=[
        ("cat", ohe, categorical_cols),
        ("num", scaler, numeric_cols),
    ],
    remainder="drop"
)

X_train = preprocess.fit_transform(X_train_raw)
X_val   = preprocess.transform(X_val_raw)
X_test  = preprocess.transform(X_test_raw)

# 列名回填
ohe_names = list(preprocess.named_transformers_["cat"].get_feature_names_out(categorical_cols)) if categorical_cols else []
all_names = ohe_names + numeric_cols

X_train = pd.DataFrame(X_train, columns=all_names, index=X_train_raw.index)
X_val   = pd.DataFrame(X_val,   columns=all_names, index=X_val_raw.index)
X_test  = pd.DataFrame(X_test,  columns=all_names, index=X_test_raw.index)

y_train = y_train.reset_index(drop=True)
y_val   = y_val.reset_index(drop=True)
y_test  = y_test.reset_index(drop=True)

X_train.head(3)

In [ ]:
# Provide some explanations on why you believe it is important to perform this data transformation and its impacts
data_transformation_1_explanations = """
What & Why（做什么/为什么）：

Categorical → One-Hot：occupation, birth_country, credit_mix, min_amount_payment, payment_behaviour, last_1_loan_type。

Numerical → 标准化：对已清洗与工程后的数值特征做 StandardScaler（仅用 train 拟合），提升模型稳定性与可比性。

Drop/Keep：保留 dti 与 emi_to_income，删除原始 outstanding_debt 以降低共线性（但保留其 log1p__outstanding_debt 作为稳健金额刻画）。

Impacts（影响）：

OHE 让树模型/线性模型都能直接使用类别变量；

标准化对 LR/LinearSVC/SVM/KNN 等尤为重要；

使用训练集统计量拟合，避免信息泄漏；

移除 outstanding_debt 原值，保留 dti + log1p__outstanding_debt，控制共线性同时保留金额信号。
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_transformation_1_explanations', value=data_transformation_1_explanations)

### D.3 Data Transformation <put_name_here>

In [ ]:
# <Student to fill this section>

In [ ]:
# <Student to fill this section and then remove this comment>
data_transformation_2_explanations = """
Provide some explanations on why you believe it is important to perform this data transformation and its impacts
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_transformation_2_explanations', value=data_transformation_2_explanations)

### D.4 Data Transformation <put_name_here>


In [ ]:
# <Student to fill this section and then remove this comment>

In [ ]:
# <Student to fill this section and then remove this comment>
data_transformation_3_explanations = """
Provide some explanations on why you believe it is important to perform this data transformation and its impacts
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_transformation_3_explanations', value=data_transformation_3_explanations)

### D.n Fixing "\<describe_issue_here\>"

> You can add more cells related to data preparation in this section

In [ ]:
# <Student to fill this section and then remove this comment>

In [ ]:
# <Student to fill this section and then remove this comment>
data_transformation_n_explanations = """
Provide some explanations on why you believe it is important to perform this data transformation and its impacts
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_transformation_n_explanations', value=data_transformation_n_explanations)

---
## E. Save Datasets

> Do not change this code

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL

try:
  X_train.to_csv(at.folder_path / 'X_train.csv', index=False)
  y_train.to_csv(at.folder_path / 'y_train.csv', index=False)

  X_val.to_csv(at.folder_path / 'X_val.csv', index=False)
  y_val.to_csv(at.folder_path / 'y_val.csv', index=False)

  X_test.to_csv(at.folder_path / 'X_test.csv', index=False)
  y_test.to_csv(at.folder_path / 'y_test.csv', index=False)
except Exception as e:
  print(e)